In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from praxis.YoloPointingTransformer import TransformerPointingTrainer
from omegaconf import DictConfig, OmegaConf
from torch.utils.data import DataLoader, WeightedRandomSampler, Subset
from praxis.pc_dataset import *
from pathlib import Path
import logging
import torch
import os
from pathlib import Path

In [65]:
def setup_dataloader(cfg: DictConfig) -> DataLoader:

    DEVICE = cfg.DEVICE
    if DEVICE == "cpu" and cfg.task == "train":
        logging.warning("Training DeePoint with CPU takes a long time.")
    if cfg.task == "test" and cfg.verbose is not True and cfg.shrink_rate != 1:
        logging.warning(
            "Using only part of test dataset. You should set `shrink_rate=1` except for speeding up the performance for visualization"
        )

    assert not (
        cfg.task == "test" and cfg.ckpt is None
    ), "checkpoint should be specified for evaluation"
    
    keypoints_path = Path(os.getcwd()).parent.parent / cfg.keypoints_root
    train_ds = PCDataset(keypoints_path, cfg)
    
    train_dl = DataLoader(
        train_ds,
        batch_size=cfg.hardware.bs,
        #sampler=train_sampler,
        num_workers=cfg.hardware.nworkers,
        persistent_workers=True if cfg.hardware.nworkers != 0 else False,
    )
    
    return train_dl

In [66]:
def main_func():
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    cfg = OmegaConf.load("../conf/pcmodel.yaml")
    cfg.DEVICE = DEVICE
    dl = setup_dataloader(cfg)
    print(cfg)

In [67]:
main_func()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/davidtung/Documents/GitHub/data/keypoints/triangulation.pickle'